In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use('Solarize_Light2')
%matplotlib inline
print(pd.__version__)

1.0.5


In [24]:
%load_ext watermark
%watermark -n -u -v -iv -w

Last updated: Wed Jan 06 2021

Python implementation: CPython
Python version       : 3.8.2
IPython version      : 7.16.1

pandas    : 1.0.5
numpy     : 1.18.5
matplotlib: 3.2.2
seaborn   : 0.10.1

Watermark: 2.1.0



Exercise
*Please use Python 

The attached CSV file lists the customer, date, and dollar value of orders placed at a store in 2017. The actual gender and predicted gender of each customer is also provided.

Complete each of the following activities in a jupyter notebook using Python. Send back only your notebook file and please do not zip it. Please do not exclude $0 orders.

 

A) Assemble a dataframe with one row per customer and the following columns:

* customer_id * gender * most_recent_order_date * order_count (number of orders placed by this customer)

Sort the dataframe by customer_id ascending and display the first 10 rows.

B) Plot the count of orders per week for the store.

C) Compute the mean order value for gender 0 and for gender 1. Do you think the difference is significant? Justify your choice of method.

D) Generate a confusion matrix for the gender predictions of customers in this dataset. You should assume that there is only one gender prediction for each customer. What does the confusion matrix tell you about the quality of the predictions?

E) Describe one of your favorite tools or techniques and give a small example of how it's helped you solve a problem. Limit your answer to one paragraph, and please be specific.

For each question, state any considerations or assumptions you made. 

## A) Assemble a dataframe with one row per customer and the following columns:
- customer_id * gender * most_recent_order_date * order_count (number of orders placed by this customer)
- Sort the dataframe by customer_id ascending and display the first 10 rows.

In [12]:
df_data = pd.read_csv('screening_exercise_orders_v201810.csv')
df_data.head(n=1)

,customer_id,gender,date,value,predicted_gender
0,1000,0,2017-01-01 00:11:31,198.5,1


In [3]:
df_data_to_work = df_data.copy()
df_data_to_work.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13471 entries, 0 to 13470
Data columns (total 5 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   customer_id       13471 non-null  int64  
 1   gender            13471 non-null  int64  
 2   date              13471 non-null  object 
 3   value             13471 non-null  float64
 4   predicted_gender  13471 non-null  int64  
dtypes: float64(1), int64(3), object(1)
memory usage: 526.3+ KB


In [4]:
df_data_to_work['date'] = pd.to_datetime(df_data_to_work.date)
display(df_data_to_work.info())

grouped_customer_id = df_data_to_work.groupby('customer_id') 

assert grouped_customer_id.filter(lambda x: len(np.unique(x['gender'])) > 1).shape[0] == 0
assert grouped_customer_id.filter(lambda x: len(np.unique(x['predicted_gender'])) > 1).shape[0] == 0

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13471 entries, 0 to 13470
Data columns (total 5 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   customer_id       13471 non-null  int64         
 1   gender            13471 non-null  int64         
 2   date              13471 non-null  datetime64[ns]
 3   value             13471 non-null  float64       
 4   predicted_gender  13471 non-null  int64         
dtypes: datetime64[ns](1), float64(1), int64(3)
memory usage: 526.3 KB


None

In [5]:
df_data_to_work_agg = df_data_to_work.groupby(by = ['customer_id','gender']).agg(
    most_recent_order_date=pd.NamedAgg(column='date', aggfunc='max'),
    order_count=pd.NamedAgg(column='date', aggfunc='count')).reset_index()

df_data_to_work_agg.sort_values(by = ['customer_id'],inplace=True)
df_data_to_work_agg.head(n=10)

,customer_id,gender,most_recent_order_date,order_count
0,1000,0,2017-01-01 00:11:31,1
1,1001,0,2017-01-01 00:29:56,1
2,1002,1,2017-02-19 21:35:31,3
3,1003,1,2017-04-26 02:37:20,4
4,1004,0,2017-01-01 03:11:54,1
5,1005,1,2017-12-16 01:39:27,2
6,1006,1,2017-05-09 15:27:20,3
7,1007,0,2017-01-01 15:59:50,1
8,1008,0,2017-12-17 05:47:48,3
9,1009,1,2017-01-01 19:27:17,1


In [22]:
df_data_to_work.set_index( ['date']).resample('W').agg({"customer_id":np.count_nonzero})
#df_data_to_work.head(n=18)

,customer_id
date,
2017-01-01,14
2017-01-08,180
2017-01-15,264
2017-01-22,405
2017-01-29,233
2017-02-05,211
2017-02-12,195
2017-02-19,248
2017-02-26,315
